In [1]:
import re
import nltk
import numpy as np
import pandas as pd
from ast import literal_eval
from pymystem3 import Mystem
from string import punctuation

In [2]:
data_path = '../parsed/processed.csv'
save_path = '../parsed/stemmed.csv'

In [3]:
df = pd.read_csv(data_path, parse_dates=['dtm'], converters={"tags": literal_eval})
df

,id,data-story-long,rating,meta-rating,author_id,comments,dtm,views,saves,author_name,title,tags,text
0,7337284,True,7727.0,7884:157,1002684,479,2020-04-02 15:12:18+03:00,NaN,181,ksenobianinSanta,Обычная история об обычном негодяе,"[Коронавирус, Безответственность, Длиннопост, ...",столичный ресторатор илья эстер сказочно прово...
1,7337282,True,15.0,25:10,2008181,6,2020-04-02 15:11:33+03:00,NaN,4,Merey202,Гэбриел Бирн,"[Биография, Ирландия, Актеры и актрисы, Длинно...",ирландец нести черный пес печаль плечо бе...
2,7337254,True,28.0,39:11,2522374,2,2020-04-02 15:01:49+03:00,NaN,11,Cat.Cat,Ещё немного о трибунате в Римской империи,"[[моё], Cat_cat, История, Рим, Римская империя...",обсуждение пикабу пост особенность заклады...
3,7337245,True,6.0,18:12,3073815,13,2020-04-02 14:58:46+03:00,NaN,0,vitaminys2,Эд Бакканский. Враг-государство,"[Украина, Коронавирус, Государство, Длиннопост...",сегодня видеть пол нервно озираться сторо...
4,7337246,True,21.0,28:7,3156206,8,2020-04-02 14:58:20+03:00,NaN,5,ZloyElvis,Кнопки для проведения викторины,"[[моё], Arduino, Электроника, Самоделки, Викто...",приветствовать данный длиннопост изготовлени...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
135371,7185786,True,63.0,72:9,2660666,23,2020-01-24 13:40:51+03:00,NaN,4,napyc,Был ли спорт на русском флоте или хотя бы физп...,"[[моё], Флот, История, Моряки, Спорт, Интересн...",иметься вид время завершение парусный эпоха ...
135372,7185784,True,2080.0,2333:253,2456769,487,2020-01-24 13:38:54+03:00,NaN,65,RevizorMinsk,Тайный покупатель проверил пикабу,"[[моё], Проверка, Пикабу, Тайный покупатель, Р...",честь 100 го выпуск история тайный покупатель...
135373,7185777,True,1.0,10:9,2940548,15,2020-01-24 13:35:47+03:00,NaN,0,SiBARS,Ангел-хранитель для ребенка,"[[моё], Ангел-Хранитель, Провидение, Судьба, И...",навевать пост предопределенность свыше весь ка...
135374,7185769,True,8712.0,8882:170,2423817,782,2020-01-24 13:28:04+03:00,NaN,170,akustic.nhk,Как мы письмо в администрацию президента напис...,"[[моё], Угольная пыль, Поджог, Приморский край...",интересный дело твориться прекрасный город ...


In [4]:
class Porter:
    PERFECTIVEGROUND =  re.compile(u"((ив|ивши|ившись|ыв|ывши|ывшись)|((?<=[ая])(в|вши|вшись)))$")
    REFLEXIVE = re.compile(u"(с[яь])$")
    ADJECTIVE = re.compile(u"(ее|ие|ые|ое|ими|ыми|ей|ий|ый|ой|ем|им|ым|ом|его|ого|ему|ому|их|ых|ую|юю|ая|яя|ою|ею)$")
    PARTICIPLE = re.compile(u"((ивш|ывш|ующ)|((?<=[ая])(ем|нн|вш|ющ|щ)))$")
    VERB = re.compile(u"((ила|ыла|ена|ейте|уйте|ите|или|ыли|ей|уй|ил|ыл|им|ым|ен|ило|ыло|ено|ят|ует|уют|ит|ыт|ены|ить|ыть|ишь|ую|ю)|((?<=[ая])(ла|на|ете|йте|ли|й|л|ем|н|ло|но|ет|ют|ны|ть|ешь|нно)))$")
    NOUN = re.compile(u"(а|ев|ов|ие|ье|е|иями|ями|ами|еи|ии|и|ией|ей|ой|ий|й|иям|ям|ием|ем|ам|ом|о|у|ах|иях|ях|ы|ь|ию|ью|ю|ия|ья|я)$")
    RVRE = re.compile(u"^(.*?[аеиоуыэюя])(.*)$")
    DERIVATIONAL = re.compile(u".*[^аеиоуыэюя]+[аеиоуыэюя].*ость?$")
    DER = re.compile(u"ость?$")
    SUPERLATIVE = re.compile(u"(ейше|ейш)$")
    I = re.compile(u"и$")
    P = re.compile(u"ь$")
    NN = re.compile(u"нн$")
    
    @staticmethod
    def stem(word):
        word = word.replace(u'ё', u'е')
        m = re.match(Porter.RVRE, word)
        if m and m.groups():
            pre = m.group(1)
            rv = m.group(2)
            temp = Porter.PERFECTIVEGROUND.sub('', rv, 1)
            if temp == rv:
                rv = Porter.REFLEXIVE.sub('', rv, 1)
                temp = Porter.ADJECTIVE.sub('', rv, 1)
                if temp != rv:
                    rv = temp
                    rv = Porter.PARTICIPLE.sub('', rv, 1)
                else:
                    temp = Porter.VERB.sub('', rv, 1)
                    if temp == rv:
                        rv = Porter.NOUN.sub('', rv, 1)
                    else:
                        rv = temp
            else:
                rv = temp
            
            rv = Porter.I.sub('', rv, 1)
            if re.match(Porter.DERIVATIONAL, rv):
                rv = Porter.DER.sub('', rv, 1)

            temp = Porter.P.sub('', rv, 1)
            if temp == rv:
                rv = Porter.SUPERLATIVE.sub('', rv, 1)
                rv = Porter.NN.sub(u'н', rv, 1)
            else:
                rv = temp
            word = pre+rv
        return word

In [5]:
def stem_text(text):
    return " ".join(Porter.stem(w) for w in text.split())

In [6]:
df["text"] = df.apply(lambda row: stem_text(row["text"]), axis=1)  # override text

In [7]:
df.to_csv(save_path, encoding='utf-8', index=False)